In [1]:
import numpy as np

In [115]:
a = [1,2,3,4,5]
b = ['-', '*', "/", '+']
muChance = 0.05

In [116]:
def generatePoulation(a,b,n):
    aCopy = a.copy()
    pop = []
    operandPop = []
    for i in range(n):
        operandChrom = []
        for j in range(len(a)):
            if j != 0:
                bb = np.random.randint(len(b))
                operandChrom.append(b[bb])

        operandPop.append(operandChrom)
        chrom = []
        lena = len(a)
        for j in range(lena):
            aa = np.random.randint(len(a))
            chrom.append(a[aa])
            del a[aa]
        pop.append(chrom)
        a = aCopy.copy()
    return pop, operandPop

In [117]:
def operandMutation(operand, muChance, b):
    for i in range(len(operand)):
        if np.random.random()>=float(1-muChance):
            operand[i] = b[np.random.randint(len(b))]
    return operand

In [118]:
def ChromeMutation(chrome, muChance):
    for i in range(len(chrome)):
        if np.random.random() >= float(1-muChance):
            g1 = np.random.randint(len(chrome))
            g2 = np.random.randint(len(chrome))
            buf = chrome[g2]
            chrome[g2] = chrome[g1]
            chrome[g1] = buf
    return chrome

In [119]:
pop, popOperand = generatePoulation(a,b,3)

UnboundLocalError: local variable 'lena' referenced before assignment

In [92]:
print(pop, popOperand)
operandMutation(popOperand[1], muChance, b)

[[4, 5, 1, 3, 2], [5, 1, 4, 2, 3], [2, 1, 4, 3, 5]] [['+', '-', '+'], ['/', '-', '-'], ['-', '+', '+']]


['/', '-', '-']

In [93]:
ChromeMutation(pop[2], muChance)

[2, 1, 4, 3, 5]

In [100]:
def crossOver(chrom1, chrom2, operand1, operand2):
    
    lenop = int(len(operand1)/2)
    operand = operand1[0:lenop]
    operand.extend(operand2[lenop:])
    chrom = []
    for i in range(len(chrom1)):
        chrom.append(chrom2[chrom1[i]-1])
    
    return chrom, operand

In [101]:
c,o = crossOver(pop[0],pop[1], popOperand[0], popOperand[1])
print(c,o)

[2, 3, 5, 4, 1] ['+', '-', '-']


In [102]:
pop[1]

[5, 1, 4, 2, 3]

In [103]:
pop[0]

[4, 5, 1, 3, 2]

In [108]:
def compute(chrom, operand):
    string = str(chrom[0])
    for i in range(len(operand)):
        string += str(operand[i])
        string += str(chrom[i+1])
        
    return eval(string)

In [109]:
compute(pop[0],popOperand[0])

11

In [110]:
print(pop[0],popOperand[0])

[4, 5, 1, 3, 2] ['+', '-', '+']
